In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU device found:')
    for device in physical_devices:
        print(device)
else:
    print('No GPU devices found.')

In [ ]:
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
with tf.device('/GPU:0'):
    import tensorflow as tf

    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)

    # Set the path to your dataset directory
    dataset_dir = '/content/drive/MyDrive/Files/Spectrograms_Frequency'

    # Define the image dimensions and batch size
    image_size = (558, 217)
    batch_size = 32

    # Load the dataset using image_dataset_from_directory
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_dir,
        validation_split=0.2,
        subset='training',
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
        color_mode='rgb'
    )

    validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_dir,
        validation_split=0.2,
        subset='validation',
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
        color_mode='rgb'
    )

    # Print class names
    class_names = train_dataset.class_names
    print("Class names:", class_names)

    # Normalize pixel values and configure dataset for performance
    AUTOTUNE = tf.data.AUTOTUNE
    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
    train_dataset = train_dataset.map(lambda x, y: (x / 255.0, y))  # Normalize pixel values to [0, 1]
    validation_dataset = validation_dataset.map(lambda x, y: (x / 255.0, y))

    # Define the model architecture
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    # Print the model summary
    model.summary()

In [ ]:
with tf.device('/GPU:0'):

    # Train the model
    history = model.fit(train_dataset, validation_data=validation_dataset, epochs=80)

    # Evaluate the model
    loss, accuracy = model.evaluate(validation_dataset)
    print("Validation loss:", loss)
    print("Validation accuracy:", accuracy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# model training plots
epochs = np.arange(1,11)
plt.subplot(1,2,1)
plt.plot(epochs,history.history['loss'],color='r')
plt.title('Model loss (sparse categorical crossentropy)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.tight_layout()
plt.subplot(1,2,2)
plt.plot(epochs,history.history['accuracy'],color='g')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.tight_layout()
plt.show()

In [ ]:
# Save the trained model
# model.save('./Models\Basic_CNN')

In [ ]:
tf.config.experimental.clear_memory()